# Ratterdam Place Field Repetition
## Notebook to analyze place field repetition phenomenology and the factors that drive it

In [1]:
import numpy as np, random, json, pickle, datetime, copy, socket, os, sys
from scipy.stats import sem
import matplotlib.colors as colors
from importlib import reload
import matplotlib.pyplot as plt
from PIL import Image

from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.widgets import Button

if socket.gethostname() == 'Tolman':
    codeDirBase = 'C:\\Users\\whockei1\\Google Drive'
elif socket.gethostname() == 'DESKTOP-BECTOJ9':
    codeDirBase = 'C:\\Users\\whock\\Google Drive'

sys.path.insert(0, codeDirBase + '\\KnierimLab\\Ratterdam\\Code')
sys.path.insert(0, codeDirBase + '\\Python_Code\\KLab\\mts_analysis')
import utility_fx as util
import ratterdam_ParseBehavior as pBehav
import ratterdam_CoreDataStructures as core
import ratterdam_PermutationTests as Perm
import ratterdam_Directionality as Dir
import ratterdam_visBasic as Vis
from ratterdam_Defaults import *

%qtconsole --style native

In [3]:
exp = "RFD5"
dayCode = f"R765{exp}\\"
figpath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\R765{exp}\\"
if socket.gethostname() == 'Tolman':
    datafile = f"E:\\Ratterdam\\R765\\R765{exp}\\"
elif socket.gethostname() == 'DESKTOP-BECTOJ9':
    datafile = "C:\\Users\\whock\\Google Drive\\KnierimLab\\Ratterdam\\Data\\R765_ephys\\R765RFD7\\"
behav = core.BehavioralData(datafile, exp, 1)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()

In [4]:
unit = core.UnitData("TT4\\cl-maze1.4", datafile, exp, alleyBounds, alleyVisits, txtVisits, position, ts)
unit.loadData_raw()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:306: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:310: RuntimeWarning: invalid value encount

In [5]:
allAlleyVisits = []
for alley,visitList in alleyVisits.items():
    visits = [(alley+1,i, v[0], v[1]) for i,v in enumerate(visitList)]
    allAlleyVisits.extend(visits)
allAlleyVisits = sorted(allAlleyVisits, key = lambda x: x[2])

In [297]:
'''
Routine to create subplots for each visit to an alley +/- some # seconds
Spikes heat coded by their order, gives sense of where he entered. Could be better
'''

alley = 12
tBefore, tAfter = [3e6]*2 # time in ms before/after visit to add to visit data
numPlots = len(unit.alleys[alley])
fig, axis = plt.subplots(int(np.ceil(numPlots/8)), 8)
bounds = Dir.extractCorners(alleyBounds[alley-1])
for vNum, visit in enumerate(unit.alleys[alley]):
    ax = fig.axes[vNum]
    start, end= alleyVisits[alley-1][vNum][0] - tBefore, alleyVisits[alley-1][vNum][1] + tAfter
    
    side = Dir.checkVisitEntrySide(visit['occs'], bounds)
    
    sTs = util.getTsinInverval(unit.spikes, start, end)
    spikes = util.getPosFromTs(sTs[:,0],position)
    
    oTs = util.getTsinInverval(position, start, end)
    occs = util.getPosFromTs(oTs[:,0],position)
    
    ratPlot.drawAlleyBounds(ax,alley)
    
    ax.scatter(spikes[:,0], spikes[:,1],c=range(spikes.shape[0]), s=25, alpha=0.4)
    ax.plot(occs[:,0], occs[:,1])
    ax.scatter(occs[-1,0], occs[-1,1], marker='^')
    
    if side == 'NE':
        ax.scatter(alleyBounds[alley-1][0][0], alleyBounds[alley-1][1][1], c='g',s=100,marker='^')
    elif side == 'SW':
        ax.scatter(alleyBounds[alley-1][0][0], alleyBounds[alley-1][1][0], c='g',s=100,marker='^')
        

    
    extend = 40
    ax.set_xlim([alleyBounds[alley-1][0][0]-extend, alleyBounds[alley-1][0][1]+extend])
    ax.set_ylim([alleyBounds[alley-1][1][0]-extend, alleyBounds[alley-1][1][1]+extend])

In [6]:
def getPreviousAlley(alley, visitNum):
    currVisitIdx = [i for i,x in enumerate(allAlleyVisits) if x[0] == alley and x[1] == visitNum][0]
    prevAlley = allAlleyVisits[currVisitIdx-1][0]
    return prevAlley

In [7]:
def getNextAlley(alley, visitNum):
    currVisitIdx = [i for i,x in enumerate(allAlleyVisits) if x[0] == alley and x[1] == visitNum][0]
    nextAlley = allAlleyVisits[currVisitIdx+1][0]
    return nextAlley

In [378]:
"""
Turn into alley
"""
turnType = "Retrospective"
alley = 10
rms = {side:{turn:np.empty((0,30)) for turn in ['left', 'right']} for side in ['NE', 'SW']}
idx  = {side:{turn:[] for turn in ['left', 'right']} for side in ['NE', 'SW']}
sideMarkers = []

bounds = Dir.extractCorners(alleyBounds[alley-1])
for vNum, visit in enumerate(unit.alleys[alley]):

    side = Dir.checkVisitEntrySide(visit['occs'], bounds)
    
    prevAlley = getPreviousAlley(alley, vNum) #returns an alley
    
    if side == 'NE':
        sideMarkers.append("g")
        if alleyBounds[prevAlley-1][0][0] < alleyBounds[alley-1][0][0]:
            turn = 'right'
        else:
            turn = 'left'
            
    elif side == 'SW':
        sideMarkers.append("r")
        if alleyBounds[prevAlley-1][0][0] < alleyBounds[alley-1][0][0]:
            turn = 'left'
        else:
            turn = 'right'
            
    rms[side][turn] = np.vstack((rms[side][turn], unit.alleys[alley][vNum]['ratemap1d']))
    idx[side][turn].append(vNum)
    
    
    

In [373]:
"""
Based on turn out of alley 
"""
turnType = "Prospective"
alley = 10
rms = {side:{turn:np.empty((0,30)) for turn in ['left', 'right']} for side in ['NE', 'SW']}
idx  = {side:{turn:[] for turn in ['left', 'right']} for side in ['NE', 'SW']}
sideMarkers = []

bounds = Dir.extractCorners(alleyBounds[alley-1])
for vNum, visit in enumerate(unit.alleys[alley]):

    side = Dir.checkVisitEntrySide(visit['occs'], bounds)
    
    if vNum < len(unit.alleys[alley])-1:
    
        nextAlley = getNextAlley(alley, vNum) #returns an alley

        if side == 'NE':
            sideMarkers.append("g")
            if alleyBounds[nextAlley-1][0][0] > alleyBounds[alley-1][0][0]:
                turn = 'right'
            else:
                turn = 'left'

        elif side == 'SW':
            sideMarkers.append("r")
            if alleyBounds[nextAlley-1][0][0] > alleyBounds[alley-1][0][0]:
                turn = 'left'
            else:
                turn = 'right'

        rms[side][turn] = np.vstack((rms[side][turn], unit.alleys[alley][vNum]['ratemap1d']))
        idx[side][turn].append(vNum)
    
    
    

In [379]:
fig, ax = plt.subplots(2,2)

# get the common max for imshow
allmats = np.empty((0,30))
for side in ['NE', 'SW']:
    for turn in ['left', 'right']:
        allmats = np.vstack((allmats, rms[side][turn]))
mymax = util.calcSmartMax(allmats, cutoff=0.97, scale=2, bins=100)

i=0
for side in ['NE', 'SW']:
    for turn in ['left', 'right']:
        fig.axes[i].imshow(rms[side][turn], vmax=mymax, cmap=mycmap, interpolation='None', aspect='auto')
        fig.axes[i].set_title(f"{side} {turn}")
        fig.axes[i].set_yticks(range(len(idx[side][turn])))
        fig.axes[i].set_yticklabels(idx[side][turn])
        
        i+=1
plt.suptitle(f"{turnType} {alley}")

Text(0.5,0.98,'Retrospective 10')

In [338]:
plt.figure()
styleDict = {'NE':{'left':["b","--"], "right":["b", "-"]}, "SW":{"left":["r","--"], "right":["r", "-"]}}
for side in ["NE", "SW"]:
    for turn in ["left", "right"]:
        avg = np.nanmean(rms[side][turn], axis=0)
        err = sem(rms[side][turn], axis=0, nan_policy="omit")
        plt.plot(avg,color=styleDict[side][turn][0], linestyle=styleDict[side][turn][1])
        plt.fill_between(range(len(avg)), avg-err, avg+err, color=styleDict[side][turn][0], alpha=0.5)
plt.legend(["NE left", "NE right", "SW left", "SW right"], fontsize=24)
plt.title(f"{turnType} {alley}")

Text(0.5,1,'Retrospective 12')

### Live animation of activity

In [2]:
ratPlot = Vis.BasicRateMaps()

In [28]:
def plot_all_alleyBounds(ax):
    for alley in range(1,18):
        ratPlot.drawAlleyBounds(ax, alley)

In [28]:
class RatterdamAnimation():
    
    def __init__(self, vfile):
        self.fig, self.ax = plt.subplots()
        #self.start_time = position[0,0]
        self.start_time = 1907446288746
        self.increment_time = 5e6 # size of window
        self.offset_time = 1e6 # size of slide
        self.current_epoch_time = self.start_time # this refers to timestamp of the beginning of the currently plotted period of activity
        self.animation_finished = False
        self.paused = False
        self.cap = cv2.VideoCapture(vfile)        
    
        
    def forward(self, event):
        self.current_epoch_time += 5e6
        
    def prev(self, event):
        self.current_epoch_time -= 5e6
        
    def myprint(self, event):
        print(self.current_epoch_time)
        
    def pause(self, event):
        if self.paused == True:
            self.paused = False
        else:
            self.paused = True
            
    def parseSMIFile(self):
        """
        SMI file is generic (i.e. used beyond NL) type to store subtitles
        Neuralynx stores the NL ts of a video frame as subtitles
        So extract them into a np array
        """
        with open(datafile+"VT1.smi","r") as file:
            lines = file.readlines()
        tsLines = [line for line in lines if 'SYNC' in line] # SYNC tag being the unique one for ts data AFAIK
        pattern = re.compile("<.*?>")
        NLts = [int(re.sub(pattern,'',entry).strip()) for entry in tsLines]
        self.NLts = np.array(NLts)
        
            
    def getCurrentVideoFrame(self):
        start, end = self.current_epoch_time, self.current_epoch_time + self.increment_time
        idx, = np.where(np.logical_and(self.NLts > start, self.NLts < end))
        self.cap.set(1, idx[-1])
        ret, frame = self.cap.read()
        frame = np.flip(frame, axis=1)
        pic = Image.fromarray(frame)
        pic.thumbnail((640,480))
        frame = np.asarray(pic)
        return frame
        
            
    def visualizeSession_scatter(self):
        
        self.parseSMIFile()
        
        while not self.animation_finished:


                    self.ax.set_xlim([0,640])
                    self.ax.set_ylim([0,480])

                    #plot_all_alleyBounds(self.ax)

#                     occs = util.getTsinInverval(position, self.current_epoch_time, self.current_epoch_time + self.increment_time)
#                     occs = util.getPosFromTs(occs[:,0], position)

#                     spikes = util.getTsinInverval(unit.spikes, self.current_epoch_time, self.current_epoch_time + self.increment_time)
#                     spikes = util.getPosFromTs(spikes[:,0], position)

#                     self.ax.plot(occs[:,0], occs[:,1], color='k', alpha=0.7)
#                     self.ax.scatter(spikes[:,0], spikes[:,1],s=56,c='r',alpha=0.7)
#                     self.ax.set_title(self.current_epoch_time,fontsize=24)
                    
                    frame = self.getCurrentVideoFrame()
                    self.ax.imshow(frame[:,:,2],origin='upper')
                    if not self.paused:
                        self.current_epoch_time = self.current_epoch_time + self.offset_time

                    if self.current_epoch_time > 1907952676606: #position[-1,0]:
                        self.animation_finished = True

                    plt.pause(0.1)
                    self.ax.clear()
                    
    def visualizeSession_rm(self):
        while not self.animation_finished:  
            if not self.paused:
                self.ax.set_xlim([0,640])
                self.ax.set_ylim([0,480])
    
                plot_all_alleyBounds(self.ax)

                occs = util.getTsinInverval(position, self.current_epoch_time, self.current_epoch_time + self.increment_time)
                occs = util.getPosFromTs(occs[:,0], position)

                spikes = util.getTsinInverval(unit.spikes, self.current_epoch_time, self.current_epoch_time + self.increment_time)
                spikes = util.getPosFromTs(spikes[:,0], position)

                self.ax.plot(occs[:,0], occs[:,1], color='k', alpha=0.7)
                self.ax.scatter(spikes[:,0], spikes[:,1],s=56,c='r',alpha=0.7)
                self.ax.set_title(self.current_epoch_time,fontsize=24)

                self.current_epoch_time = self.current_epoch_time + self.offset_time

                if self.current_epoch_time > position[-1,0]:
                    self.animation_finished = True

                plt.pause(0.05)
                self.ax.clear()
                
    
    def run(self):
        
        
        self.axpause = self.fig.add_axes([0.48, 0.05, 0.1, 0.075])
        bpause = Button(self.axpause, "Pause")
        bpause.on_clicked(self.pause)
        
        self.axstore = self.fig.add_axes([0.59, 0.05, 0.1, 0.075])
        bstore = Button(self.axstore, "Print")
        bstore.on_clicked(self.myprint)
        
        self.axnext = self.fig.add_axes([0.81, 0.05, 0.1, 0.075])
        bnext = Button(self.axnext, 'Next')
        bnext.on_clicked(self.forward)
        
        self.axprev = self.fig.add_axes([0.7, 0.05, 0.1, 0.075])
        bprev = Button(self.axprev, 'Prev')
        bprev.on_clicked(self.prev)

        self.visualizeSession_scatter()

    
    

In [29]:
rattAm = RatterdamAnimation(datafile+"VT1.mpg")
rattAm.run()

NameError: name 'time' is not defined

In [41]:
#Parse .SMI file - subtitles of NL ts on video frames
with open(datafile+"VT1.smi","r") as file:
    lines = file.readlines()
tsLines = [line for line in lines if 'SYNC' in line] # SYNC tag being the unique one for NL ts data AFAIK
pattern = re.compile("<.*?>")
NLts = [int(re.sub(pattern,'',entry).strip()) for entry in tsLines]
NLts = np.array(NLts)

## Reverse Correlation and related analyses
Idea is to look at all behaviors that precede/follow a given level of activity and see if some common behavioral feature explains firing.
Based on spike-triggered averaging ideas

In [ ]:
def read_pos(datafile):
    '''open pos.p.ascii and parse into {ts:[x,y,dir]} '''
    posfile = datafile + 'Pos.p.ascii'
    with open(posfile) as posP:
        raw_position_data = posP.read().splitlines()
        on_data = posP.read().splitlines()
    position_data = {}
    for line in raw_position_data[25:]:
      data = [float(a) for a in line.split(',')]
      position_data[data[0]] = data[1:]
    return position_data

In [8]:
def computeTrialTrajectories(region, extent=2e6):

    allSpikes = {'Ahead':[], 'Behind':[], 'Full':[]}
    allPos = {'Ahead':[], 'Behind':[], 'Full':[]}

    for lookType, tBefore, tAfter in zip(["Ahead","Behind","Full"], [0, extent, extent], [extent, 0, extent]):

        if type(region) == int:
            visitLen = len(unit.alleys[region])
        elif type(region) == str:
            visitLen = len(intersectionVisits[region])
            
        for vNum in range(visitLen):
            
            if type(region) == int:
                start, end= alleyVisits[region-1][vNum][0] - tBefore, alleyVisits[region-1][vNum][1] + tAfter
            elif type(region) == str:
                start, end= intersectionVisits[region][vNum][0] - tBefore, intersectionVisits[region][vNum][1] + tAfter

            sTs = util.getTsinInverval(unit.spikes, start, end)
            spikes = util.getPosFromTs(sTs[:,0],position)

            oTs = util.getTsinInverval(position, start, end)
            occs = util.getPosFromTs(oTs[:,0],position)

            spkNum = spikes.shape[0]

            allSpikes[lookType].append((spkNum/occs.shape[0])*30)
            allPos[lookType].append(occs)
            
    return allSpikes, allPos

In [31]:
for a in [2,13,9,7,11,5]:
    region = a
    extent = 2e6 # time in ms to look either ahead or behind
    allSpikes = {'Ahead':[], 'Behind':[], 'Full':[]}
    allPos = {'Ahead':[], 'Behind':[], 'Full':[]}
    fig, Axis = plt.subplots(1,3, figsize=(10,6))

    for lookType, ax, tBefore, tAfter in zip(["Ahead","Behind","Full"], fig.axes, [0, extent, extent], [extent, 0, extent]):

        if type(region) == int:
            visitLen = len(unit.alleys[region])
        elif type(region) == str:
            visitLen = len(intersectionVisits[region])
            
        for vNum in range(visitLen):
            
            if type(region) == int:
                start, end = alleyVisits[region-1][vNum][0] - tBefore, alleyVisits[region-1][vNum][1] + tAfter
            elif type(region) == str:
                start, end = intersectionVisits[region][vNum][0] - tBefore, intersectionVisits[region][vNum][1] + tAfter

            sTs = util.getTsinInverval(unit.spikes, start, end)
            spikes = util.getPosFromTs(sTs[:,0],position)

            oTs = util.getTsinInverval(position, start, end)
            occs = util.getPosFromTs(oTs[:,0],position)

            spkNum = spikes.shape[0]

            #so its rate not spikes. 
            allSpikes[lookType].append((spkNum/occs.shape[0])*30)
            allPos[lookType].append(occs)

    _max = max(allSpikes['Ahead']+allSpikes['Behind']+allSpikes['Full'])
    _min = min(allSpikes['Ahead']+allSpikes['Behind']+allSpikes['Full'])
    
        
    for i, lookType in enumerate(["Ahead", "Behind", "Full"]):
        
        allSpikes[lookType] = [mynorm(x,_min, _max) for x in allSpikes[lookType]]
        ax = fig.axes[i]
        
        #note spkNum is now rate as of 1/12/19 so these names should be changed
        for spkNum, visit in zip(allSpikes[lookType], allPos[lookType]): 
            if spkNum > 0.0*max(allSpikes[lookType]):
                ax.plot(visit[:,0], visit[:,1],c=mymap(spkNum), alpha=0.6, linestyle='-')
                ax.scatter(visit[-1,0], visit[-1,1], marker='^',color=mymap(spkNum),alpha=0.6)
                ax.scatter(visit[0,0], visit[0,1], marker='o',color=mymap(spkNum),alpha=0.6)
            
            #TODO - make drawAlleyBounds work on ints. i have code in other notebook. 
            if type(region) == int:
                ratPlot.drawAlleyBounds(ax, region)

            ax.set_title(lookType)
                
    plt.suptitle(f"Region {region}, Look dur: {extent/1e6}s, max Rate: {round(_max,2)}, velc thresh: {velocity_filter_thresh}")

In [12]:
def mynorm(x,_min,_max):
    return (x-_min)/(_max-_min)

In [148]:
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import path

intersectionList = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
intersectionTracking = {i:np.empty((0,3)) for i in intersectionList}
for intersection in intersectionList:
    p = path.Path(intersectionBounds[intersection])
    inOrNot = p.contains_points(position[:,1:])
    intersectionTracking[intersection] = position[inOrNot]

In [149]:
intersectionVisits = {i:[] for i in ['A','B','C','D','E','F','G','H','I','J','K','L']} # each list will contain ts corresponding to time of alley visit beginning
for intersection,timestamps in intersectionTracking.items():
    vts = pBehav.getVisits(timestamps[:,0],default_visit_gap)
    vts = [i for i in vts if len(i)>1]
    vts = [i for i in vts if (i[-1]-i[0])>default_visit_duration]
    visitTs = [(i[0], i[-1]) for i in vts] # vts returns list of list, each of which is all ts in a group. So first ts is the first ts of a visit, grab it
    intersectionVisits[intersection] = visitTs

In [197]:
# each visit, sep plot
alley = 12
allSpikes, allPos = computeTrialTrajectories(alley)
extend = 100
numPlots = len(allSpikes['Full'])
fig, axis = plt.subplots(int(np.ceil(numPlots/8)), 8)
bounds = Dir.extractCorners(alleyBounds[alley-1])

_max = max(allSpikes['Full'])
_min = min(allSpikes['Full'])
        
allSpikes['Full'] = [mynorm(x,_min, _max) for x in allSpikes['Full']]

for i,(spkNum, visit) in enumerate(zip(allSpikes['Full'], allPos['Full'])):
    ax = fig.axes[i]
    ax.plot(visit[:,0], visit[:,1],c=mymap(spkNum), alpha=0.5, linestyle='-')
    ax.scatter(visit[-1,0], visit[-1,1], marker='^',color=mymap(spkNum),alpha=0.6)
    ax.scatter(visit[0,0], visit[0,1], marker='o',color=mymap(spkNum),alpha=0.6)
    ax.set_xlim([alleyBounds[alley-1][0][0]-extend, alleyBounds[alley-1][0][1]+extend])
    ax.set_ylim([alleyBounds[alley-1][1][0]-extend, alleyBounds[alley-1][1][1]+extend])

    ratPlot.drawAlleyBounds(ax, alley)
    
plt.suptitle(f"{alley}")
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.0,
                    wspace=0.0)

for ax in fig.axes:
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 

    for spine in ['top', 'right', 'bottom', 'left']:
        ax.spines[spine].set_visible(False)


In [61]:
#active alleys

In [32]:
alleys = [5,7,11,13,2,9]

allSpikeDict = {a:[] for a in alleys}
allPosDict = {a:[] for a in alleys}
for a in alleys:
    allSpike, allPos = computeTrialTrajectories(a)
    allSpikeDict[a] = allSpike
    allPosDict[a] = allPos

spike_thresh = 0.2#percent max
activeTrials = []
allTrials = []
for (alley,vnum,tstart,tend) in allAlleyVisits:
    if alley in alleys:
        allTrials.append((tstart, alley, allSpikeDict[alley]['Full'][vnum], allPosDict[alley]['Full'][vnum]))
        if allSpikeDict[alley]['Full'][vnum] >= spike_thresh*max(allSpikeDict[alley]['Full']):
            activeTrials.append((tstart, alley, allSpikeDict[alley]['Full'][vnum], allPosDict[alley]['Full'][vnum]))

In [71]:
linT = {a:[[],[]] for a in alleys}
linTall = {a:[[],[]] for a in alleys}
for a in alleys:
    for (time, alley, spk, occ) in activeTrials:
        if alley == a:
            linT[a][0].append(time)
            linT[a][1].append(spk)     
            
    for (time, alley, spk, occ) in allTrials:
        if alley == a:
            linTall[a][0].append(time)
            linTall[a][1].append(spk)
            
for a in alleys:
    #plt.plot(linT[a][0],util.weird_smooth(np.asarray(linT[a][1]),2),marker='.', label=f"{a}+", linewidth=2)
    if a in [13,9,2]:
        alpha=1.0
    else:
        alpha=0.4
    plt.plot(linTall[a][0],util.weird_smooth(np.asarray(linTall[a][1]),2),alpha=alpha, marker='.', label=f"{a}")

plt.legend(fontsize=24)

In [159]:
# plot by breaking session into thirds
lookType = "Full"
for alley in [6]:
    
    fig, axes = plt.subplots(1,3)

    entryTimes = np.asarray([i[0] for i in alleyVisits[alley-1]])
    allSpikes, allPos = computeTrialTrajectories(alley)
    
    _max = max(allSpikes[lookType])
    _min = min(allSpikes[lookType])
    allSpikes[lookType] = [mynorm(x,_min, _max) for x in allSpikes[lookType]]
    
    np.linspace(position[0,0], position[-1,0],4)
    intervals = np.linspace(position[0,0], position[-1,0],4)
    
    
    for i,label in zip(range(len(intervals)-1), ['First', 'Second', 'Third']):
        ax = fig.axes[i]
        start, end = intervals[i], intervals[i+1]
        idx, = np.where(np.logical_and(entryTimes > start, entryTimes < end))
        
        
        
        for x in idx:
            visit,spkNum = allPos[lookType][x], allSpikes[lookType][x]
            ax.plot(visit[:,0], visit[:,1],c=mymap(spkNum), alpha=0.6, linestyle='-')
            ax.scatter(visit[-1,0], visit[-1,1], marker='^',color=mymap(spkNum),alpha=0.6)
            ax.scatter(visit[0,0], visit[0,1], marker='o',color=mymap(spkNum),alpha=0.6)
            ratPlot.drawAlleyBounds(ax, alley)
            
    plt.suptitle(f"Alley {alley} Session Activity Divided into Thirds, {lookType} Trajs")
        

### Trajectory based firing - Trial rate maps and their averages

In [34]:
def getAlleyOrientation(alley):
    xdiff, ydiff = alleyBounds[alley-1][1][0] - alleyBounds[alley-1][0][0], alleyBounds[alley-1][1][1] - alleyBounds[alley-1][0][1] 
    if xdiff > ydiff:
        return "horizontal"
    elif ydiff > xdiff:
        return "vertical"

In [35]:
def midpoint(p1,p2):
    return [(p1[0]+p2[0])/2, (p1[1]+p2[1])/2]

In [36]:
def getAlleySideMidpoint(alley, side):
    alleyOrientation = getAlleyOrientation(alley)
    x, X,= alleyBounds[alley-1][0] # lower case is min, upper case is max
    y, Y = alleyBounds[alley-1][1]
    if alleyOrientation == 'horizontal':
        if side == 'SW':
            midp = midpoint([x,y],[x, Y])
        elif side == 'NE':
            midp = midpoint([X,y],[X,Y])
            
    if alleyOrientation == 'vertical':
        if side == 'SW':
            midp = midpoint([x,y],[X,y])
        elif side == 'NE':
            midp = midpoint([x,Y],[X,Y])
            
    return midp      

In [37]:
def posScan(midp, p):
    x1,y1 = midp
    x2,y2 = p
    dist = math.hypot(x2 - x1, y2 - y1)
    return dist < (extent*ptsCm)


In [38]:
def extendTrajectory(seed, midpoint, direction):
    """
    """
    if direction == 'forward':
        posSlice = position[seed:]
        extendedTraj = np.asarray(list(itertools.takewhile(lambda x: posScan(midpoint, x[1:]), posSlice)))
    elif direction == 'reverse':
        posSlice = np.flip(position[0:seed+1],axis=0)
        extendedTraj = np.flip(np.asarray(list(itertools.takewhile(lambda x: posScan(midpoint, x[1:]), posSlice))),axis=0)
                
    if extendedTraj != []:
        return extendedTraj[0,0]
        print(extendedTraj[0,0])
    else:
        return None

In [39]:
def getDistBasedTrajBounds(vnum, alley):
    """
    
    """
    visitStart = util.takeClosest(position[:,0], alleyVisits[alley-1][vnum][0]) 
    visitEnd = util.takeClosest(position[:,0], alleyVisits[alley-1][vnum][1]) 
    bounds = Dir.extractCorners(alleyBounds[alley-1])
    entrySide = util.checkVisitSide(unit.alleys[alley][vnum]['occs'], bounds, "entry")
    exitSide = util.checkVisitSide(unit.alleys[alley][vnum]['occs'], bounds, "exit")
    
    midp = getAlleySideMidpoint(alley, entrySide)
    seed, = np.where(position[:,0] == visitStart)[0]
    start = extendTrajectory(seed, midp, direction='reverse')
    
    midp = getAlleySideMidpoint(alley, exitSide)
    seed, = np.where(position[:,0] == visitEnd)[0]
    end = extendTrajectory(seed, midp, direction='forward')
    
    return start, end

In [44]:
def computeSingleRM(spikes,position,alley,dim):
    '''
    Modification of the standard computeSingleRM that
    does not bin by alley coords but just by bin num
    
    Compute a single rate map from an array of spikes
    and pos of array form (ts,x,y). Not using track coords, just 
    0-max bin size in either array. Chooses which dim is long
    depending on which alley (lookup in fx)'''
    longDimBins = singleAlleyBins[0]
    shortDimBins = singleAlleyBins[1]
    v = alleyBounds[alley-1]
    x,y = v
    if (x[-1]-x[0]) > (y[-1]-y[0]):
        rbins, cbins = shortDimBins, longDimBins 
    elif (y[-1]-y[0]) > (x[-1]-x[0]):
        rbins, cbins = longDimBins, shortDimBins
    hs = np.histogram2d(spikes[:,2],spikes[:,1],bins=[rbins, cbins])[0]
    ho = np.histogram2d(position[:,2],position[:,1],bins=[rbins, cbins])[0]
    if dim == 2:
        n = (hs*np.reciprocal(ho))*30
        n[np.where(ho==0)] = np.nan
        n = util.weird_smooth(n,smoothing_2d_sigma)
        n[np.where(ho==0)] = np.nan
    elif dim == 1:
        ls,lo  = np.sum(hs,axis=util.getAxType(ho)), np.sum(ho,axis=util.getAxType(ho))
        n = (ls* np.reciprocal(lo)) * 30 
        n = util.weird_smooth(n,smoothing_1d_sigma)
        n[np.where(lo==0)] = np.nan
    return n

In [71]:
region = 15
extent = 10
alley = region
tol = 15 #with alleybounds recorded from video, youre reading them off so theres jitter. need an epsilon fo
bounds = Dir.extractCorners(alleyBounds[alley-1])

# IMPORTANT - this nested dict is unlabeled but convention is turn into the alley > turn out of alley
# so outer dict is turn in and inner dict is turn out given that turn in
trajectorySorted = {'left':{'left':[], 'right':[], 'straight':[]}, 
                    'right':{'left':[], 'right':[], 'straight':[]}, 
                    'straight':{'left':[], 'right':[], 'straight':[]}}

trajLabels = {'left':{'left':[], 'right':[], 'straight':[]}, 
                    'right':{'left':[], 'right':[], 'straight':[]}, 
                    'straight':{'left':[], 'right':[], 'straight':[]}}

for vNum, visit in enumerate(unit.alleys[region]):    
#     if type(region) == int:
#         start, end= alleyVisits[region-1][vNum][0] - tBefore, alleyVisits[region-1][vNum][1] + tAfter
#     elif type(region) == str:
#         start, end= intersectionVisits[region][vNum][0] - tBefore, intersectionVisits[region][vNum][1] + tAfter

    start, end = getDistBasedTrajBounds(vNum, alley)
    
    if (start != None) and (end != None):

        sTs = util.getTsinInverval(unit.spikes, start, end)
        spikes = util.getPosFromTs(sTs[:,0],position)
    
        oTs = util.getTsinInverval(position, start, end)
        occs = util.getPosFromTs(oTs[:,0],position)


        side = Dir.checkVisitEntrySide(visit['occs'], bounds)

        prevAlley = getPreviousAlley(region, vNum) #returns an alley
        nextAlley = getNextAlley(region, vNum)

        alleyOrientation = getAlleyOrientation(region)

        if alleyOrientation == 'vertical':
            if side == 'NE':
                if (alleyBounds[prevAlley-1][0][0] < alleyBounds[alley-1][0][0]) and (alleyBounds[prevAlley-1][0][0] < alleyBounds[alley-1][0][0]) :
                    inTurn = 'right'
                elif alleyBounds[prevAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    inTurn = 'left'
                elif alleyBounds[prevAlley]
                    inTurn = 'straight'

                if alleyBounds[prevAlley-1][0][0] < alleyBounds[alley-1][0][0]:
                    outTurn = 'right'
                elif alleyBounds[prevAlley-1][0][0] > alleyBounds[alley-1][0][0]:
                    outTurn = 'left'
                else:
                    outTurn = 'straight'

            if side == 'SW':
                if alleyBounds[prevAlley-1][0][0] < alleyBounds[alley-1][0][0]:
                    inTurn = 'left'
                elif alleyBounds[prevAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    inTurn = 'right'
                else:
                    inTurn = 'straight'

                if alleyBounds[nextAlley-1][0][1] < alleyBounds[alley-1][0][0]:
                    outTurn = 'left'
                elif alleyBounds[nextAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    outTurn = 'right'
                else:
                    outTurn = 'straight'

        elif alleyOrientation == 'horizontal':

            if side == 'NE':
                if alleyBounds[prevAlley-1][0][1] < alleyBounds[alley-1][0][1]:
                    inTurn = 'right'
                elif alleyBounds[prevAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    inTurn = 'left'
                else:
                    inTurn = 'straight'

                if alleyBounds[prevAlley-1][0][1] < alleyBounds[alley-1][0][1]:
                    outTurn = 'right'
                elif alleyBounds[prevAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    outTurn = 'left'
                else:
                    outTurn = 'straight'

            if side == 'SW':
                if alleyBounds[prevAlley-1][0][1] < alleyBounds[alley-1][0][1]:
                    inTurn = 'right'
                elif alleyBounds[prevAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    inTurn = 'left'
                else:
                    inTurn = 'straight'

                if alleyBounds[nextAlley-1][0][1] < alleyBounds[alley-1][0][1]:
                    outTurn = 'right'
                elif alleyBounds[nextAlley-1][0][1] > alleyBounds[alley-1][0][1]:
                    outTurn = 'left'
                else:
                    outTurn = 'straight'
                    
        spikes = np.column_stack((np.ones(spikes.shape[0]), spikes))
        occs = np.column_stack((np.ones(occs.shape[0]), occs))

        rm = computeSingleRM(spikes, occs, alley, 1)
        trajectorySorted[inTurn][outTurn].append(rm)
        trajLabels[inTurn][outTurn].append(vNum)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in reciprocal
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:306: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in multiply


In [72]:
fig1, ax1 = plt.subplots(3,3, figsize=(7,7))
fig2, ax2 = plt.subplots()

i=0
rmsForMax = np.empty((0, singleAlleyBins[1]))

linLegend = []

for inTurn in ['left', 'right', 'straight']:
    for outTurn in ['left', 'right', 'straight']:
        for rm in trajectorySorted[inTurn][outTurn]:
            rmsForMax = np.vstack((rm, rmsForMax))
            
mymax = util.calcSmartMax(rmsForMax)

for inTurn in ['left', 'right', 'straight']:
    for outTurn in ['left', 'right', 'straight']:
        
        cax = fig1.axes[i]
        
        rms = np.empty((0, singleAlleyBins[1]))
            
        for rm in trajectorySorted[inTurn][outTurn]:
            rms = np.vstack((rms, rm))
            
                        
        if True:
            cax.imshow(rms, vmax = mymax, aspect='auto', interpolation='none', cmap=mymap)
            cax.set_title(f"{inTurn} Turn in, {outTurn} Turn out")
            cax.set_yticks(range(len(trajLabels[inTurn][outTurn])))
            cax.set_yticklabels(trajLabels[inTurn][outTurn])

            avg = np.nanmean(rms, axis=0)
            err = sem(rms, axis=0, nan_policy="omit")

            ax2.plot(avg)
            ax2.fill_between(range(len(err)), avg+err, avg-err, alpha=0.3)
            linLegend.append(f"{inTurn}-{outTurn}")
        
        i += 1
        
ax2.legend(linLegend)        

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\axes\_base.py:3239: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.5, top=-0.5
  'bottom=%s, top=%s') % (bottom, top))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: Mean of empty slice
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=

# Spike-Triggered Boundary Envelope

In [7]:
def defineLine(p0, theta, dist=500):
    """Given a starting point and an angle,
    draw a line (segment) of length dist.
    The dist is long enough that for practical
    purposes it's an infinite "line"
    
    Assumes theta is in degrees and converts
    it to radians for math.cos,math.sin
    """
    x,y = p0
    theta = math.radians(theta)
    xnew, ynew = x + dist*math.cos(theta), y + dist*math.sin(theta)
    return [xnew, ynew]

In [51]:
p0=[460,200]
plt.plot(p0[0], p0[1], 'r.', markersize=50)
for theta in np.linspace(0, 360, num=100):
    xnew, ynew = defineLine(p0, theta)
    plt.plot([p0[0], xnew], [p0[1], ynew],'k')

In [8]:
#assumes a,b are endpoints and c is the check point
def distance(a,b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def is_between(a,b,c):
    """
    a,b are endpoints. c is point to check if its btwn
    """
    epsilon = 0.001
    return -epsilon < (distance(a, c) + distance(c, b) - distance(a, b)) < epsilon

In [9]:
def get_line(x1, y1, x2, y2):
    points = []
    issteep = abs(y2-y1) > abs(x2-x1)
    if issteep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    rev = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        rev = True
    deltax = x2 - x1
    deltay = abs(y2-y1)
    error = int(deltax / 2)
    y = y1
    ystep = None
    if y1 < y2:
        ystep = 1
    else:
        ystep = -1
    for x in range(x1, x2 + 1):
        if issteep:
            points.append((y, x))
        else:
            points.append((x, y))
        error -= deltay
        if error < 0:
            y += ystep
            error += deltax
    # Reverse the list if the coordinates were reversed
    if rev:
        points.reverse()
    return points

In [10]:
#define sides of alley n, s, e, w in camera coordinates
all_points = []
for alley in range(17):
    b = alleyBounds[alley] # for ease of typing
    ul, ll, ur, lr = [b[0][0], b[1][1]], [b[0][0], b[1][0]], [b[0][1], b[1][1]], [b[0][1], b[1][0]]
    for side in [ul, ur], [ll, lr], [ur, lr], [ul, ll]:
        x1, y1, x2, y2 = side[0][0], side[0][1], side[1][0], side[1][1]
        points = get_line(x1, y1, x2, y2)
        all_points.extend(points)
all_points = np.asarray(all_points)

In [11]:
#burst detection
minISI = 0.06
minBurstNum = 3
isis = [(a-b)/1e6 for a,b in zip(unit.spikes[1:,0], unit.spikes[:,0])]

#here define the intraburst ISI to be the smallest
#peak of the isi distribution, provided it does not
# exceed the minISI. 
c,bins = np.histogram(isis,bins=75)
if bins[0] <= minISI:
    burstISI = bins[0]
else:
    burstISI = minISI

#override burstISI for testing
burstISI = 0.04
bursts = [[unit.spikes[0,:]]]
for i, spike in enumerate(unit.spikes[1:,:]):
    if abs(spike[0] - bursts[-1][-1][0])/1e6 <= burstISI:
        bursts[-1].append(spike)
    else:
        bursts.append([spike])
        
bursts = [b for b in bursts if len(b) >= minBurstNum]
#pos at first spike of burst
initBurstPos = np.asarray([burst[0][1:] for burst in bursts])

In [47]:
def defineBurstRays(point,num=100):
    rayEndPoints = []
    for theta in np.linspace(0,360,num=num):
        rayEndPoints.append(defineLine(point, theta))
    return np.asarray(rayEndPoints)

In [20]:
#define boundary envelope for a burst
envelopes = []
for burst in initBurstPos:
    rays = defineBurstRays(burst)
    envelope = np.empty((0,2))
    for ray in rays:
        onLine = []
        for point in all_points:
            onLine.append(is_between(burst,ray,point))
        idxOnLine = [i for i,b in enumerate(onLine) if b == True]
        ptsOnLine = all_points[idxOnLine]
        if ptsOnLine.shape[0] > 0:
            closestPtOnRay = ptsOnLine[np.argmin([distance(burst, p) for p in ptsOnLine])]
            envelope = np.vstack((envelope, closestPtOnRay))
    envelopes.append(envelope)


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 